In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import os 
#import cv2
from tqdm import tqdm_notebook
from random import shuffle
import shutil
import pandas as pd

In [22]:
import keras.applications
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Input
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import optimizers

In [ ]:
import keras
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import Callback

## Callback for loss logging per epoch
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        
history = LossHistory()

## Callback for early stopping the training
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')

checkpoint = keras.callbacks.ModelCheckpoint(model_weights_checkpoint, monitor='val_loss', verbose=1, save_best_only=True, mode='max')

In [3]:
img_width, img_height = 150, 150

In [4]:
# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(img_width, img_height, 3))  # this assumes K.image_data_format() == 'channels_last'

In [6]:
#base_model = keras.applications.InceptionResNetV2(weights='imagenet', include_top=False)
#base_model = keras.applications.ResNet50(include_top=False, weights='imagenet')#, input_tensor=input_tensor)
base_model = keras.applications.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

In [7]:
# add a global spatial average pooling layer
x = base_model.output

In [23]:
# let's add a fully-connected layer
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
#x = Dropout(0.5)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(3, activation='softmax')(x)

In [24]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [25]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [26]:
# compile the model (should be done *after* setting layers to non-trainable)
#model.compile(optimizer='sgd', loss='categorical_crossentropy')
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [28]:
from keras.preprocessing.image import ImageDataGenerator

In [29]:
train_data_dir = 'C:\\Users\\rafae\\Desktop\\Coleta\\data\\rgb-subset\\train'
validation_data_dir = 'C:\\Users\\rafae\\Desktop\\Coleta\\data\\rgb-subset\\validation'
nb_train_samples = 2500
nb_validation_samples = 560
epochs = 50
batch_size = 32
samples_per_epoch= 5000

In [30]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    #shear_range=0.2,
    zoom_range=0.2,
    #horizontal_flip=True
    width_shift_range=0.2,
    height_shift_range=0.2)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [31]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 2500 images belonging to 3 classes.
Found 560 images belonging to 3 classes.


In [32]:
# train the model on the new data for a few epochs
#model.fit_generator(...)

# fine-tune the model
model.fit_generator(
    train_generator,
    #samples_per_epoch=nb_train_samples,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks = [checkpoint, history,early_stopping ])

Epoch 1/50
78/78 [==============================] - ETA: 13:51 - loss: 0.6343 - acc: 0.66 - ETA: 7:37 - loss: 0.6305 - acc: 0.6667 - ETA: 5:31 - loss: 0.6266 - acc: 0.666 - ETA: 4:28 - loss: 0.6333 - acc: 0.666 - ETA: 3:49 - loss: 0.6340 - acc: 0.666 - ETA: 3:23 - loss: 0.6335 - acc: 0.666 - ETA: 3:04 - loss: 0.6345 - acc: 0.666 - ETA: 2:50 - loss: 0.6337 - acc: 0.666 - ETA: 2:38 - loss: 0.6354 - acc: 0.666 - ETA: 2:29 - loss: 0.6341 - acc: 0.666 - ETA: 2:21 - loss: 0.6333 - acc: 0.666 - ETA: 2:14 - loss: 0.6345 - acc: 0.666 - ETA: 2:08 - loss: 0.6335 - acc: 0.666 - ETA: 2:03 - loss: 0.6348 - acc: 0.666 - ETA: 1:58 - loss: 0.6342 - acc: 0.666 - ETA: 1:53 - loss: 0.6338 - acc: 0.666 - ETA: 1:49 - loss: 0.6342 - acc: 0.666 - ETA: 1:46 - loss: 0.6340 - acc: 0.666 - ETA: 1:42 - loss: 0.6333 - acc: 0.666 - ETA: 1:39 - loss: 0.6328 - acc: 0.666 - ETA: 1:36 - loss: 0.6329 - acc: 0.666 - ETA: 1:33 - loss: 0.6332 - acc: 0.666 - ETA: 1:31 - loss: 0.6332 - acc: 0.666 - ETA: 1:28 - loss: 0.6334 - 

78/78 [==============================] - ETA: 1:34 - loss: 0.6182 - acc: 0.666 - ETA: 1:33 - loss: 0.6142 - acc: 0.666 - ETA: 1:32 - loss: 0.6156 - acc: 0.666 - ETA: 1:31 - loss: 0.6120 - acc: 0.666 - ETA: 1:30 - loss: 0.6097 - acc: 0.666 - ETA: 1:29 - loss: 0.6100 - acc: 0.666 - ETA: 1:27 - loss: 0.6099 - acc: 0.666 - ETA: 1:26 - loss: 0.6093 - acc: 0.666 - ETA: 1:25 - loss: 0.6077 - acc: 0.666 - ETA: 1:17 - loss: 0.6077 - acc: 0.666 - ETA: 1:16 - loss: 0.6081 - acc: 0.666 - ETA: 1:16 - loss: 0.6087 - acc: 0.666 - ETA: 1:15 - loss: 0.6086 - acc: 0.666 - ETA: 1:14 - loss: 0.6090 - acc: 0.666 - ETA: 1:13 - loss: 0.6092 - acc: 0.666 - ETA: 1:12 - loss: 0.6101 - acc: 0.666 - ETA: 1:11 - loss: 0.6106 - acc: 0.666 - ETA: 1:10 - loss: 0.6106 - acc: 0.666 - ETA: 1:09 - loss: 0.6103 - acc: 0.666 - ETA: 1:08 - loss: 0.6103 - acc: 0.666 - ETA: 1:07 - loss: 0.6097 - acc: 0.666 - ETA: 1:06 - loss: 0.6094 - acc: 0.666 - ETA: 1:05 - loss: 0.6098 - acc: 0.666 - ETA: 1:04 - loss: 0.6095 - acc: 0.666 -

78/78 [==============================] - ETA: 1:35 - loss: 0.5798 - acc: 0.666 - ETA: 1:33 - loss: 0.5894 - acc: 0.666 - ETA: 1:32 - loss: 0.5853 - acc: 0.666 - ETA: 1:31 - loss: 0.5850 - acc: 0.666 - ETA: 1:30 - loss: 0.5834 - acc: 0.666 - ETA: 1:29 - loss: 0.5829 - acc: 0.666 - ETA: 1:27 - loss: 0.5837 - acc: 0.666 - ETA: 1:26 - loss: 0.5858 - acc: 0.666 - ETA: 1:25 - loss: 0.5872 - acc: 0.666 - ETA: 1:24 - loss: 0.5880 - acc: 0.666 - ETA: 1:23 - loss: 0.5874 - acc: 0.666 - ETA: 1:21 - loss: 0.5867 - acc: 0.666 - ETA: 1:20 - loss: 0.5867 - acc: 0.666 - ETA: 1:19 - loss: 0.5866 - acc: 0.666 - ETA: 1:18 - loss: 0.5873 - acc: 0.666 - ETA: 1:16 - loss: 0.5882 - acc: 0.666 - ETA: 1:15 - loss: 0.5890 - acc: 0.666 - ETA: 1:14 - loss: 0.5888 - acc: 0.666 - ETA: 1:13 - loss: 0.5890 - acc: 0.666 - ETA: 1:11 - loss: 0.5886 - acc: 0.666 - ETA: 1:10 - loss: 0.5887 - acc: 0.666 - ETA: 1:09 - loss: 0.5885 - acc: 0.666 - ETA: 1:08 - loss: 0.5881 - acc: 0.666 - ETA: 1:06 - loss: 0.5884 - acc: 0.666 -

78/78 [==============================] - ETA: 1:34 - loss: 0.5788 - acc: 0.677 - ETA: 1:33 - loss: 0.5754 - acc: 0.677 - ETA: 1:32 - loss: 0.5679 - acc: 0.673 - ETA: 1:31 - loss: 0.5702 - acc: 0.671 - ETA: 1:30 - loss: 0.5663 - acc: 0.672 - ETA: 1:28 - loss: 0.5697 - acc: 0.671 - ETA: 1:27 - loss: 0.5711 - acc: 0.671 - ETA: 1:26 - loss: 0.5710 - acc: 0.670 - ETA: 1:25 - loss: 0.5694 - acc: 0.670 - ETA: 1:24 - loss: 0.5681 - acc: 0.670 - ETA: 1:16 - loss: 0.5634 - acc: 0.670 - ETA: 1:15 - loss: 0.5629 - acc: 0.670 - ETA: 1:15 - loss: 0.5624 - acc: 0.670 - ETA: 1:14 - loss: 0.5640 - acc: 0.670 - ETA: 1:13 - loss: 0.5667 - acc: 0.670 - ETA: 1:12 - loss: 0.5667 - acc: 0.669 - ETA: 1:11 - loss: 0.5669 - acc: 0.669 - ETA: 1:10 - loss: 0.5674 - acc: 0.669 - ETA: 1:09 - loss: 0.5681 - acc: 0.669 - ETA: 1:08 - loss: 0.5681 - acc: 0.669 - ETA: 1:07 - loss: 0.5689 - acc: 0.669 - ETA: 1:06 - loss: 0.5701 - acc: 0.669 - ETA: 1:05 - loss: 0.5709 - acc: 0.669 - ETA: 1:04 - loss: 0.5704 - acc: 0.669 -

78/78 [==============================] - ETA: 1:35 - loss: 0.5337 - acc: 0.718 - ETA: 1:33 - loss: 0.5554 - acc: 0.697 - ETA: 1:33 - loss: 0.5591 - acc: 0.691 - ETA: 1:31 - loss: 0.5665 - acc: 0.684 - ETA: 1:30 - loss: 0.5677 - acc: 0.683 - ETA: 1:29 - loss: 0.5704 - acc: 0.680 - ETA: 1:28 - loss: 0.5674 - acc: 0.681 - ETA: 1:26 - loss: 0.5619 - acc: 0.683 - ETA: 1:25 - loss: 0.5606 - acc: 0.687 - ETA: 1:24 - loss: 0.5588 - acc: 0.687 - ETA: 1:23 - loss: 0.5577 - acc: 0.687 - ETA: 1:21 - loss: 0.5613 - acc: 0.685 - ETA: 1:20 - loss: 0.5598 - acc: 0.687 - ETA: 1:19 - loss: 0.5608 - acc: 0.687 - ETA: 1:18 - loss: 0.5613 - acc: 0.688 - ETA: 1:16 - loss: 0.5605 - acc: 0.689 - ETA: 1:15 - loss: 0.5604 - acc: 0.690 - ETA: 1:14 - loss: 0.5593 - acc: 0.692 - ETA: 1:13 - loss: 0.5591 - acc: 0.693 - ETA: 1:12 - loss: 0.5591 - acc: 0.692 - ETA: 1:10 - loss: 0.5584 - acc: 0.693 - ETA: 1:09 - loss: 0.5583 - acc: 0.691 - ETA: 1:08 - loss: 0.5575 - acc: 0.692 - ETA: 1:07 - loss: 0.5574 - acc: 0.692 -

...

78/78 [==============================] - ETA: 1:35 - loss: 0.4826 - acc: 0.802 - ETA: 1:34 - loss: 0.4798 - acc: 0.786 - ETA: 1:33 - loss: 0.4888 - acc: 0.770 - ETA: 1:31 - loss: 0.4831 - acc: 0.773 - ETA: 1:30 - loss: 0.4836 - acc: 0.775 - ETA: 1:29 - loss: 0.4841 - acc: 0.774 - ETA: 1:28 - loss: 0.4902 - acc: 0.767 - ETA: 1:26 - loss: 0.4876 - acc: 0.773 - ETA: 1:25 - loss: 0.4917 - acc: 0.765 - ETA: 1:24 - loss: 0.4925 - acc: 0.762 - ETA: 1:23 - loss: 0.4890 - acc: 0.768 - ETA: 1:21 - loss: 0.4889 - acc: 0.766 - ETA: 1:20 - loss: 0.4893 - acc: 0.765 - ETA: 1:19 - loss: 0.4877 - acc: 0.770 - ETA: 1:18 - loss: 0.4866 - acc: 0.772 - ETA: 1:16 - loss: 0.4854 - acc: 0.772 - ETA: 1:15 - loss: 0.4849 - acc: 0.772 - ETA: 1:14 - loss: 0.4847 - acc: 0.772 - ETA: 1:13 - loss: 0.4850 - acc: 0.774 - ETA: 1:11 - loss: 0.4828 - acc: 0.777 - ETA: 1:10 - loss: 0.4849 - acc: 0.773 - ETA: 1:09 - loss: 0.4825 - acc: 0.777 - ETA: 1:08 - loss: 0.4835 - acc: 0.776 - ETA: 1:07 - loss: 0.4855 - acc: 0.775 -

78/78 [==============================] - ETA: 1:35 - loss: 0.5016 - acc: 0.750 - ETA: 1:34 - loss: 0.5068 - acc: 0.755 - ETA: 1:32 - loss: 0.5018 - acc: 0.750 - ETA: 1:31 - loss: 0.5054 - acc: 0.752 - ETA: 1:30 - loss: 0.5055 - acc: 0.747 - ETA: 1:29 - loss: 0.5011 - acc: 0.755 - ETA: 1:28 - loss: 0.4991 - acc: 0.758 - ETA: 1:26 - loss: 0.5017 - acc: 0.756 - ETA: 1:25 - loss: 0.4966 - acc: 0.761 - ETA: 1:24 - loss: 0.4942 - acc: 0.764 - ETA: 1:23 - loss: 0.4910 - acc: 0.767 - ETA: 1:21 - loss: 0.4863 - acc: 0.770 - ETA: 1:20 - loss: 0.4832 - acc: 0.774 - ETA: 1:19 - loss: 0.4819 - acc: 0.776 - ETA: 1:18 - loss: 0.4822 - acc: 0.775 - ETA: 1:16 - loss: 0.4853 - acc: 0.770 - ETA: 1:15 - loss: 0.4819 - acc: 0.773 - ETA: 1:14 - loss: 0.4821 - acc: 0.774 - ETA: 1:13 - loss: 0.4797 - acc: 0.778 - ETA: 1:11 - loss: 0.4798 - acc: 0.779 - ETA: 1:10 - loss: 0.4789 - acc: 0.780 - ETA: 1:09 - loss: 0.4785 - acc: 0.779 - ETA: 1:08 - loss: 0.4789 - acc: 0.779 - ETA: 1:06 - loss: 0.4803 - acc: 0.776 -

78/78 [==============================] - ETA: 1:34 - loss: 0.4512 - acc: 0.833 - ETA: 1:34 - loss: 0.4695 - acc: 0.791 - ETA: 1:33 - loss: 0.4687 - acc: 0.791 - ETA: 1:31 - loss: 0.4583 - acc: 0.804 - ETA: 1:30 - loss: 0.4625 - acc: 0.802 - ETA: 1:29 - loss: 0.4651 - acc: 0.798 - ETA: 1:28 - loss: 0.4667 - acc: 0.793 - ETA: 1:26 - loss: 0.4709 - acc: 0.791 - ETA: 1:25 - loss: 0.4694 - acc: 0.795 - ETA: 1:24 - loss: 0.4681 - acc: 0.794 - ETA: 1:23 - loss: 0.4711 - acc: 0.794 - ETA: 1:22 - loss: 0.4720 - acc: 0.790 - ETA: 1:20 - loss: 0.4744 - acc: 0.790 - ETA: 1:19 - loss: 0.4763 - acc: 0.790 - ETA: 1:18 - loss: 0.4765 - acc: 0.788 - ETA: 1:13 - loss: 0.4824 - acc: 0.786 - ETA: 1:12 - loss: 0.4800 - acc: 0.785 - ETA: 1:11 - loss: 0.4782 - acc: 0.784 - ETA: 1:10 - loss: 0.4788 - acc: 0.784 - ETA: 1:09 - loss: 0.4768 - acc: 0.785 - ETA: 1:08 - loss: 0.4754 - acc: 0.787 - ETA: 1:07 - loss: 0.4744 - acc: 0.787 - ETA: 1:05 - loss: 0.4748 - acc: 0.787 - ETA: 1:04 - loss: 0.4750 - acc: 0.787 -

78/78 [==============================] - ETA: 1:35 - loss: 0.4224 - acc: 0.833 - ETA: 1:34 - loss: 0.4281 - acc: 0.833 - ETA: 1:33 - loss: 0.4387 - acc: 0.819 - ETA: 1:32 - loss: 0.4427 - acc: 0.817 - ETA: 1:30 - loss: 0.4410 - acc: 0.822 - ETA: 1:29 - loss: 0.4459 - acc: 0.814 - ETA: 1:28 - loss: 0.4491 - acc: 0.814 - ETA: 1:26 - loss: 0.4563 - acc: 0.802 - ETA: 1:25 - loss: 0.4495 - acc: 0.809 - ETA: 1:24 - loss: 0.4505 - acc: 0.809 - ETA: 1:23 - loss: 0.4529 - acc: 0.804 - ETA: 1:22 - loss: 0.4548 - acc: 0.803 - ETA: 1:20 - loss: 0.4568 - acc: 0.800 - ETA: 1:19 - loss: 0.4575 - acc: 0.800 - ETA: 1:18 - loss: 0.4560 - acc: 0.801 - ETA: 1:16 - loss: 0.4573 - acc: 0.798 - ETA: 1:15 - loss: 0.4580 - acc: 0.798 - ETA: 1:14 - loss: 0.4621 - acc: 0.796 - ETA: 1:13 - loss: 0.4594 - acc: 0.799 - ETA: 1:12 - loss: 0.4586 - acc: 0.801 - ETA: 1:10 - loss: 0.4583 - acc: 0.801 - ETA: 1:09 - loss: 0.4562 - acc: 0.804 - ETA: 1:08 - loss: 0.4570 - acc: 0.803 - ETA: 1:07 - loss: 0.4573 - acc: 0.803 -

78/78 [==============================] - ETA: 1:35 - loss: 0.4722 - acc: 0.781 - ETA: 1:34 - loss: 0.4489 - acc: 0.807 - ETA: 1:07 - loss: 0.4525 - acc: 0.788 - ETA: 1:12 - loss: 0.4459 - acc: 0.799 - ETA: 1:15 - loss: 0.4404 - acc: 0.814 - ETA: 1:16 - loss: 0.4389 - acc: 0.822 - ETA: 1:17 - loss: 0.4478 - acc: 0.815 - ETA: 1:17 - loss: 0.4440 - acc: 0.813 - ETA: 1:17 - loss: 0.4449 - acc: 0.813 - ETA: 1:17 - loss: 0.4453 - acc: 0.813 - ETA: 1:17 - loss: 0.4443 - acc: 0.813 - ETA: 1:16 - loss: 0.4458 - acc: 0.812 - ETA: 1:15 - loss: 0.4466 - acc: 0.811 - ETA: 1:14 - loss: 0.4467 - acc: 0.813 - ETA: 1:14 - loss: 0.4494 - acc: 0.809 - ETA: 1:13 - loss: 0.4451 - acc: 0.811 - ETA: 1:12 - loss: 0.4446 - acc: 0.811 - ETA: 1:11 - loss: 0.4420 - acc: 0.814 - ETA: 1:10 - loss: 0.4444 - acc: 0.812 - ETA: 1:09 - loss: 0.4457 - acc: 0.809 - ETA: 1:08 - loss: 0.4426 - acc: 0.813 - ETA: 1:07 - loss: 0.4429 - acc: 0.813 - ETA: 1:05 - loss: 0.4446 - acc: 0.811 - ETA: 1:04 - loss: 0.4435 - acc: 0.812 -

In [63]:
from keras.preprocessing import image

#img_path = 'E:\\MestradoData\\preprocessed\\all\\01\\44_1_rgb_191.png'
#img_path = 'E:\\MestradoData\\preprocessed\\all\\06\\44_6_rgb_191.png'
img_path = 'E:\\MestradoData\\preprocessed\\all\\10\\41_10_rgb_191.png'

#img_path = 'E:\\MestradoData\\preprocessed\\all\\01\\42_1_rgb_191.png'
#img_path = 'E:\\MestradoData\\preprocessed\\all\\06\\42_6_rgb_191.png'
#img_path = 'E:\\MestradoData\\preprocessed\\all\\10\\42_10_rgb_191.png'

img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

In [60]:
import datetime
a = datetime.datetime.now()
result = model.predict(x)
b = datetime.datetime.now()
print(b-a, result)

0:00:00.271595 [[8.2045525e-01 1.3249708e-11 1.7954473e-01]]


In [62]:
import datetime
a = datetime.datetime.now()
result = model.predict(x)
b = datetime.datetime.now()
print(b-a, result)

0:00:00.075515 [[6.2774462e-01 4.0403837e-11 3.7225533e-01]]


In [64]:
import datetime
a = datetime.datetime.now()
result = model.predict(x)
b = datetime.datetime.now()
print(b-a, result)

0:00:00.468661 [[5.8992170e-03 1.0613114e-15 9.9410075e-01]]
